In [21]:
!pip install kaggle_environments
import kaggle_environments
from kaggle_environments import make, evaluate, utils, agent
import numpy as np
import random
import time


## Versions

In [22]:
#@ title Heuristic Agent
def heuristic(obs, config):
    #config is dict: {'rows': 6, 'columns': 7, 'inarow': 4}

    ################################
    # Imports and helper functions #
    ################################

    import numpy as np
    import random

    # constants
    ROWS = config.rows
    COLUMNS = config.columns
    CNCTX = config.inarow
    A = 2       #2 threes
    B = 200     #10 fours
    C = -1      #-1 opp-threes
    D = -100    #-10opp-fours   

    # lookahead depth:
    N_STEPS = 2

    # Gets board at next step if agent drops piece in selected column
    def drop_piece(grid, col, mark, config):
        next_grid = grid.copy()
        for row in range(ROWS-1, -1, -1):
            if next_grid[row][col] == 0:
                break
        next_grid[row][col] = mark
        return next_grid

    # Helper function for get_heuristic: checks if window satisfies heuristic conditions
    def check_window(window, num_discs, piece, config):
        return (window.count(piece) == num_discs and window.count(0) == CNCTX-num_discs)

    # Helper function for get_heuristic: counts number of windows satisfying specified heuristic conditions
    def count_windows(grid, num_discs, piece, config):
        num_windows = 0
        # horizontal
        for row in range(ROWS):
            for col in range(COLUMNS-(CNCTX-1)):
                window = list(grid[row, col:col+CNCTX])
                if check_window(window, num_discs, piece, config):
                    num_windows += 1
        # vertical
        for row in range(ROWS-(CNCTX-1)):
            for col in range(COLUMNS):
                window = list(grid[row:row+CNCTX, col])
                if check_window(window, num_discs, piece, config):
                    num_windows += 1
        # positive diagonal
        for row in range(ROWS-(CNCTX-1)):
            for col in range(COLUMNS-(CNCTX-1)):
                window = list(grid[range(row, row+CNCTX), range(col, col+CNCTX)])
                if check_window(window, num_discs, piece, config):
                    num_windows += 1
        # negative diagonal
        for row in range(CNCTX-1, ROWS):
            for col in range(COLUMNS-(CNCTX-1)):
                window = list(grid[range(row, row-CNCTX, -1), range(col, col+CNCTX)])
                if check_window(window, num_discs, piece, config):
                    num_windows += 1
        return num_windows

    # Helper function for minimax: calculates value of heuristic for grid
    def get_heuristic(grid, mark, config, ):
        num_threes = count_windows(grid, 3, mark, config) #A
        num_fours = count_windows(grid, 4, mark, config)  #B
        num_threes_opp = count_windows(grid, 3, mark%2+1, config) #C
        num_fours_opp = count_windows(grid, 4, mark%2+1, config)  #D

        score = A*num_threes + B*num_fours + C*num_threes_opp + D*num_fours_opp
        return score

    # Helper function for minimax: checks if agent or opponent has four in a row in the window
    def is_terminal_window(window, config):
        return window.count(1) == CNCTX or window.count(2) == CNCTX

    # Helper function for minimax: checks if game has ended
    def is_terminal_node(grid, config):
        # Check for draw 
        if list(grid[0, :]).count(0) == 0:
            return True
        # Check for win: horizontal, vertical, or diagonal
        # horizontal 
        for row in range(ROWS):
            for col in range(COLUMNS-(CNCTX-1)):
                window = list(grid[row, col:col+CNCTX])
                if is_terminal_window(window, config):
                    return True
        # vertical
        for row in range(ROWS-(CNCTX-1)):
            for col in range(COLUMNS):
                window = list(grid[row:row+CNCTX, col])
                if is_terminal_window(window, config):
                    return True
        # positive diagonal
        for row in range(ROWS-(CNCTX-1)):
            for col in range(COLUMNS-(CNCTX-1)):
                window = list(grid[range(row, row+CNCTX), range(col, col+CNCTX)])
                if is_terminal_window(window, config):
                    return True
        # negative diagonal
        for row in range(CNCTX-1, ROWS):
            for col in range(COLUMNS-(CNCTX-1)):
                window = list(grid[range(row, row-CNCTX, -1), range(col, col+CNCTX)])
                if is_terminal_window(window, config):
                    return True
        return False

    # Minimax implementation was here:
    def minimax(node, depth, maximizingPlayer, mark, config):
        if depth == 0:
            return get_heuristic(node, mark, config)
        if is_terminal_node(node, config):
            return get_heuristic(node, mark, config)
        
        valid_moves = [c for c in range(COLUMNS) if node[0][c] == 0]
        if maximizingPlayer:
            value = -np.Inf
            for col in valid_moves:
                child = drop_piece(node, col, mark, config)
                value = max(value, minimax(child, depth-1, False, mark, config))
            return value
        
        else: #minimizing player
            value = np.Inf
            for col in valid_moves:
                child = drop_piece(node, col, mark%2+1, config)
                value = min(value, minimax(child, depth-1, True, mark, config))
            return value

    # Uses minimax to calculate value of dropping piece in selected column
    def score_move(grid, col, mark, config, nsteps):
        next_grid = drop_piece(grid, col, mark, config)
        score = minimax(next_grid, nsteps-1, False, mark, config) 
        return score

    #########################
    # Agent makes selection #
    #########################

    # Get list of valid moves
    valid_moves = [c for c in range(COLUMNS) if obs.board[c] == 0]

    # Convert the board to a 2D grid
    grid = np.asarray(obs.board).reshape(ROWS, COLUMNS)

    # Use the heuristic to assign a score to each possible board in the next step
    scores = dict(zip(valid_moves, [score_move(grid, col, obs.mark, config, N_STEPS) for col in valid_moves]))

    # Get a list of columns (moves) that maximize the heuristic
    max_cols = [key for key in scores.keys() if scores[key] == max(scores.values())]

    # Select at random from the maximizing columns
    return random.choice(max_cols)

In [23]:
#@title Alternate Heuristic Agent
def test_agent(obs, config):
    ################################
    # Imports and helper functions #
    ################################

    import numpy as np
    import random

    # constants
    ROWS = config.rows
    COLUMNS = config.columns
    CNCTX = config.inarow

    # lookahead depth:
    N_STEPS = 2

    # Variable depth:
    #def get_depth(num_moves):
        # Assign N_STEPS according to how many nodes in next step
    #    if num_moves >= 6 : # 6,7
    #        depth=2
    #    elif num_moves >= 4 : #4,5
    #        depth=3
    #    elif num_moves >= 2 : #2,3
    #        depth=4   
    #    else: #num_moves == 1 :
    #        depth=5   
    #    return depth

    #heuristics:    
    A =  10         #2       threes
    B =  1000       #200, 10 fours
    C = -15         #-1      opp-threes
    D = -100        #-10,-100 opp-fours

    # Gets board at next step if agent drops piece in selected column
    def drop_piece(grid, col, mark):
        next_grid = grid.copy()
        for row in range(ROWS-1, -1, -1):
            if next_grid[row][col] == 0:
                break
        next_grid[row][col] = mark
        return next_grid

    # Helper function for get_heuristic: checks if window satisfies heuristic conditions
    def check_window(window, num_discs, piece):
        return (window.count(piece) == num_discs and window.count(0) == CNCTX-num_discs)

    # Helper function for get_heuristic: counts number of windows satisfying specified heuristic conditions
    def count_windows(grid, num_discs, piece, counts):
        #num_threes, num_fours, num_threes_opp, num_fours_opp = 0
        # horizontal
        for row in range(ROWS):
            for col in range(COLUMNS-(CNCTX-1)):
                window = list(grid[row, col:col+CNCTX])
                if check_window(window, num_discs, piece):
                    #num_threes += 1
                    counts[0] += 1
        # vertical
        for row in range(ROWS-(CNCTX-1)):
            for col in range(COLUMNS):
                window = list(grid[row:row+CNCTX, col])
                if check_window(window, num_discs, piece):
                    #num_fours += 1
                    counts[1] += 1
        # positive diagonal
        for row in range(ROWS-(CNCTX-1)):
            for col in range(COLUMNS-(CNCTX-1)):
                window = list(grid[range(row, row+CNCTX), range(col, col+CNCTX)])
                if check_window(window, num_discs, piece):
                    #num_threes_opp += 1
                    counts[2] += 1
        # negative diagonal
        for row in range(CNCTX-1, ROWS):
            for col in range(COLUMNS-(CNCTX-1)):
                window = list(grid[range(row, row-CNCTX, -1), range(col, col+CNCTX)])
                if check_window(window, num_discs, piece):
                    #num_fours_opp += 1
                    counts[3] += 1
        
        #return (num_threes, num_fours, num_threes_opp, num_fours_opp)
        return counts

    # Helper function for minimax: calculates value of heuristic for grid
    def get_heuristic(grid, mark, counts):
        num_threes = counts[0] #count_windows(grid, 3, mark, config) #A
        num_fours = counts[1] #count_windows(grid, 4, mark, config)  #B
        num_threes_opp = counts[2] #count_windows(grid, 3, mark%2+1, config) #C
        num_fours_opp = counts[3] #count_windows(grid, 4, mark%2+1, config)  #D
        score = A*num_threes + B*num_fours + C*num_threes_opp + D*num_fours_opp
        return score

    # Helper function for minimax: checks if agent or opponent has four in a row in the window
    def is_terminal_window(window):
        return window.count(1) == CNCTX or window.count(2) == CNCTX

    # Helper function for minimax: checks if game has ended
    def is_terminal_node(grid, counts):
        #num_threes, num_fours, num_threes_opp, num_fours_opp = 0
        #counts = (num_threes, num_fours, num_threes_opp, num_fours_opp)
        # Check for draw 
        if list(grid[0, :]).count(0) == 0:
            return True, counts
        # Check for win: horizontal, vertical, or diagonal
        # horizontal 
        for row in range(ROWS):
            for col in range(COLUMNS-(CNCTX-1)):
                window = list(grid[row, col:col+CNCTX])
                if check_window(window, num_discs, piece):
                    #num_threes += 1
                    counts[0] += 1
                if is_terminal_window(window):
                    return True, counts
        # vertical
        for row in range(ROWS-(CNCTX-1)):
            for col in range(COLUMNS):
                window = list(grid[row:row+CNCTX, col])
                if check_window(window, num_discs, piece):
                    #num_fours += 1
                    counts[1] += 1
                if is_terminal_window(window):
                    return True, counts
        # positive diagonal
        for row in range(ROWS-(CNCTX-1)):
            for col in range(COLUMNS-(CNCTX-1)):
                window = list(grid[range(row, row+CNCTX), range(col, col+CNCTX)])
                if check_window(window, num_discs, piece):
                    #num_threes_opp += 1
                    counts[2] += 1
                if is_terminal_window(window):
                    return True, counts
        # negative diagonal
        for row in range(CNCTX-1, ROWS):
            for col in range(COLUMNS-(CNCTX-1)):
                window = list(grid[range(row, row-CNCTX, -1), range(col, col+CNCTX)])
                if check_window(window, num_discs, piece):
                    #num_fours_opp += 1
                    counts[3] += 1
                if is_terminal_window(window):
                    return True, counts
        return False, counts

    # Minimax implementation:
    def minimax(node, depth, maximizingPlayer, mark):
        counts = (0,0,0,0) #threes, fours, threes_opp, fours_opp
        is_terminal, counts = is_terminal_node(node, counts)
        if depth == 0 or is_terminal:
            return get_heuristic(node, mark, counts)
        
        valid_moves = [c for c in range(COLUMNS) if node[0][c] == 0]
        if maximizingPlayer: # = self
            value = -np.Inf
            for col in valid_moves:
                child = drop_piece(node, col, mark)
                value = max(value, minimax(child, depth-1, False, mark))
            return value
        
        else:
            value = np.Inf
            for col in valid_moves:
                child = drop_piece(node, col, mark%2+1)
                value = min(value, minimax(child, depth-1, True, mark))
            return value

    # Uses minimax to calculate value of dropping piece in selected column
    def score_move(grid, col, mark, nsteps):
        next_grid = drop_piece(grid, col, mark)
        score = minimax(next_grid, nsteps-1, False, mark) 
        return score

    #########################
    # Agent makes selection #
    #########################

    # Get list of valid moves
    valid_moves = [c for c in range(COLUMNS) if obs.board[c] == 0]

    # Assign N_STEPS according to how many valid_moves
    n_steps = N_STEPS#get_depth(len(valid_moves))
    
    # Convert the board to a 2D grid
    grid = np.asarray(obs.board).reshape(ROWS, COLUMNS)

    # Use the heuristic to assign a score to each possible board in the next step
    scores = dict(zip(valid_moves, [score_move(grid, col, obs.mark, n_steps) for col in valid_moves]))

    # Get a list of columns (moves) that maximize the heuristic
    max_cols = [key for key in scores.keys() if scores[key] == max(scores.values())]

    # Select at random from the maximizing columns
    return random.choice(max_cols)

### 5) Alpha-Beta pruning
If you decide to use the minimax code from the tutorial, you might like to add [**alpha-beta pruning**](https://en.wikipedia.org/wiki/Alpha%E2%80%93beta_pruning) to decrease the computation time (i.e., get the minimax algorithm to run much faster!).  In this case, "alpha" and "beta" to refer to two values that are maintained while the algorithm is running, that help to identify early stopping conditions.  

Without alpha-beta pruning, minimax evaluates each leaf node.  With alpha-beta pruning, minimax only evaluates nodes that could provide information that affects the agent's choice of action.  Put another way, it identifies nodes that could not possibly affect the final result and avoids evaluating them.

function minimax(node, depth, maximizingPlayer) is
    if depth = 0 or node is a terminal node then
        return the heuristic value of node
    if maximizingPlayer then
        value := −∞
        for each child of node do
            value := max(value, minimax(child, depth − 1, FALSE))
        return value
    else (* minimizing player *)
        value := +∞
        for each child of node do
            value := min(value, minimax(child, depth − 1, TRUE))
        return value

In [24]:
#@title my_agent = AlphaBeta (tree pruner)
def my_agent(obs, config):
#### AlphaBeta Agent

    ################################
    # Imports and helper functions #
    ################################
    
    import numpy as np
    import random
    
    # constants
    ROWS = config.rows
    COLUMNS = config.columns
    CNCTX = config.inarow

    # lookahead depth:
    N_STEPS = 2
    
    #heuristic:    
    A = 2   #2 threes
    B = 200  #10 fours
    C = -1  #-1 opp-threes
    D = -100 #-10opp-fours

    # Gets board at next step if agent drops piece in selected column
    def drop_piece(grid, col, mark, config):
        next_grid = grid.copy()
        for row in range(ROWS-1, -1, -1):
            if next_grid[row][col] == 0:
                break
        next_grid[row][col] = mark
        return next_grid

    # Helper function for get_heuristic: checks if window satisfies heuristic conditions
    def check_window(window, num_discs, piece, config):
        return (window.count(piece) == num_discs and window.count(0) == CNCTX-num_discs)

    # Helper function for get_heuristic: counts number of windows satisfying specified heuristic conditions
    def count_windows(grid, num_discs, piece, config):
        num_windows = 0
        # horizontal
        for row in range(ROWS):
            for col in range(COLUMNS-(CNCTX-1)):
                window = list(grid[row, col:col+CNCTX])
                if check_window(window, num_discs, piece, config):
                    num_windows += 1
        # vertical
        for row in range(ROWS-(CNCTX-1)):
            for col in range(COLUMNS):
                window = list(grid[row:row+CNCTX, col])
                if check_window(window, num_discs, piece, config):
                    num_windows += 1
        # positive diagonal
        for row in range(ROWS-(CNCTX-1)):
            for col in range(COLUMNS-(CNCTX-1)):
                window = list(grid[range(row, row+CNCTX), range(col, col+CNCTX)])
                if check_window(window, num_discs, piece, config):
                    num_windows += 1
        # negative diagonal
        for row in range(CNCTX-1, ROWS):
            for col in range(COLUMNS-(CNCTX-1)):
                window = list(grid[range(row, row-CNCTX, -1), range(col, col+CNCTX)])
                if check_window(window, num_discs, piece, config):
                    num_windows += 1
        return num_windows
    
    # Helper function for minimax: calculates value of heuristic for grid
    def get_heuristic(grid, mark, config):
        num_threes = count_windows(grid, 3, mark, config) #A
        num_fours = count_windows(grid, 4, mark, config)  #B
        num_threes_opp = count_windows(grid, 3, mark%2+1, config) #C
        num_fours_opp = count_windows(grid, 4, mark%2+1, config)  #D
        score = A*num_threes + B*num_fours + C*num_threes_opp + D*num_fours_opp
        return score

    # Helper function for minimax: checks if agent or opponent has four in a row in the window
    def is_terminal_window(window, config):
        return window.count(1) == CNCTX or window.count(2) == CNCTX

    # Helper function for minimax: checks if game has ended
    def is_terminal_node(grid, config):
        # Check for draw 
        if list(grid[0, :]).count(0) == 0:
            return True
        # Check for win: horizontal, vertical, or diagonal
        # horizontal 
        for row in range(ROWS):
            for col in range(COLUMNS-(CNCTX-1)):
                window = list(grid[row, col:col+CNCTX])
                if is_terminal_window(window, config):
                    return True
        # vertical
        for row in range(ROWS-(CNCTX-1)):
            for col in range(COLUMNS):
                window = list(grid[row:row+CNCTX, col])
                if is_terminal_window(window, config):
                    return True
        # positive diagonal
        for row in range(ROWS-(CNCTX-1)):
            for col in range(COLUMNS-(CNCTX-1)):
                window = list(grid[range(row, row+CNCTX), range(col, col+CNCTX)])
                if is_terminal_window(window, config):
                    return True
        # negative diagonal
        for row in range(CNCTX-1, ROWS):
            for col in range(COLUMNS-(CNCTX-1)):
                window = list(grid[range(row, row-CNCTX, -1), range(col, col+CNCTX)])
                if is_terminal_window(window, config):
                    return True
        return False

    # Minimax implementation with pruning:
    def alphabeta(node, depth, alpha, beta, maximizingPlayer, mark, config):
        is_terminal = is_terminal_node(node, config)
        valid_moves = [c for c in range(COLUMNS) if node[0][c] == 0]
        if depth == 0 or is_terminal:
            return get_heuristic(node, mark, config)

        if maximizingPlayer:
            value = -np.Inf
            for col in valid_moves:
                child = drop_piece(node, col, mark, config)
                value = max(value, alphabeta(child, depth-1, alpha, beta, False, mark, config))
                alpha = max(alpha, value)
                if alpha >= beta:
                    break
            return value
        
        else:
            value = np.Inf
            for col in valid_moves:
                child = drop_piece(node, col, mark%2+1, config)
                value = min(value, alphabeta(child, depth-1, alpha, beta, True, mark, config))
                beta = min(beta,value)
                if alpha >= beta:
                    break 
            return value
    
    # Uses alphabeta to calculate value of dropping piece in selected column
    def score_move(grid, col, mark, config, nsteps):
        next_grid = drop_piece(grid, col, mark, config)
        score = alphabeta(next_grid, nsteps-1,
                          -1e6, 1e6, 
                          False, mark, config) 
        return score
    
    #########################
    # Agent makes selection #
    #########################
    
    # Get list of valid moves
    valid_moves = [c for c in range(COLUMNS) if obs.board[c] == 0]
    
    # Convert the board to a 2D grid
    grid = np.asarray(obs.board).reshape(ROWS, COLUMNS)
    
    # Use the heuristic to assign a score to each possible board in the next step
    scores = dict(zip(valid_moves, [score_move(grid, col, obs.mark, config, N_STEPS) for col in valid_moves]))
    
    # Get a list of columns (moves) that maximize the heuristic
    max_cols = [key for key in scores.keys() if scores[key] == max(scores.values())]
    
    # Select at random from the maximizing columns
    return random.choice(max_cols)

##Tests

In [25]:
def get_win_percentages(agent1, agent2, n_rounds=10):
    # Use default Connect Four setup
    import numpy as np
    config = {'rows': 6, 'columns': 7, 'inarow': 4}
    # Agent 1 goes first (roughly) half the time          
    outcomes = evaluate("connectx", [agent1, agent2], config, [], n_rounds//2)
    # Agent 2 goes first (roughly) half the time      
    outcomes += [[b,a] for [a,b] in evaluate("connectx", [agent2, agent1], config, [], n_rounds-n_rounds//2)]
    print("Agent 1 Win Percentage:", np.round(outcomes.count([1,-1])/len(outcomes), 2))
    print("Agent 2 Win Percentage:", np.round(outcomes.count([-1,1])/len(outcomes), 2))
    print("Number of Invalid Plays by Agent 1:", outcomes.count([None, 0]))
    print("Number of Invalid Plays by Agent 2:", outcomes.count([0, None]))

In [26]:
# Create the game environment
env = make("connectx")

# Two random agents play one game round
env.run([my_agent, leaner_Agent])

# Show the game
env.render(mode="ipython")

In [27]:
#env.play([None,my_agent])

In [28]:
round( 428/60, 3 )

7.133

In [29]:
start_time = time.time()
n_rounds= 100
get_win_percentages(agent1=heuristic, agent2=heuristic, n_rounds=n_rounds)
print ("Total time taken: {} seconds (per round: {} seconds)".format(round(time.time() - start_time, 1), 
                                                                     round((time.time() - start_time)/n_rounds,3)))

Agent 1 Win Percentage: 0.49
Agent 2 Win Percentage: 0.42
Number of Invalid Plays by Agent 1: 0
Number of Invalid Plays by Agent 2: 0
Total time taken: 217.1 seconds (per round: 2.171 seconds)


In [30]:
start_time = time.time()
n_rounds= 100
get_win_percentages(agent1=my_agent, agent2=my_agent, n_rounds=n_rounds)
print ("Total time taken: {} seconds (per round: {} seconds)".format(round(time.time() - start_time, 1), 
                                                                     round((time.time() - start_time)/n_rounds,3)))

Agent 1 Win Percentage: 0.53
Agent 2 Win Percentage: 0.41
Number of Invalid Plays by Agent 1: 0
Number of Invalid Plays by Agent 2: 0
Total time taken: 279.3 seconds (per round: 2.793 seconds)


In [6]:
#@title Borrowed Agent 
###### Someone else's agent!! ##########
def leaner_Agent(obs, config): 
    import numpy as np

    # constants
    ROWS = config.rows
    COLUMNS = config.columns
    CNCTX = config.inarow

    def drop_piece(grid, col, mark):
        next_grid = grid.copy()
        for row in range(ROWS-1, -1, -1):
            if next_grid[row][col] == 0:
                break
        next_grid[row][col] = mark
        return next_grid

    def createNcheck_windows(grid, piece):
        my_3,my_4,op_3,op_4 = 0,0,0,0
        # horizontal
        for row in range(6):
            for col in range(4):
                window = list(grid[row, col:col+4])                
                if (window.count(piece) == 3 and window.count(0) == 1):
                    my_3 += 1
                elif (window.count(piece%2+1) == 3 and window.count(0) == 1):
                    op_3 += 1
                elif (window.count(piece) == 4 and window.count(0) == 0):
                    my_4 += 1
                elif (window.count(piece%2+1) == 4 and window.count(0) == 0):
                    op_4 += 1       
        # vertical
        for row in range(3):
            for col in range(7):
                window = list(grid[row:row+4, col])
                if (window.count(piece) == 3 and window.count(0) == 1):
                    my_3 += 1
                elif (window.count(piece%2+1) == 3 and window.count(0) == 1):
                    op_3 += 1
                elif (window.count(piece) == 4 and window.count(0) == 0):
                    my_4 += 1
                elif (window.count(piece%2+1) == 4 and window.count(0) == 0):
                    op_4 += 1
        # positive diagonal
        for row in range(3):
            for col in range(4):
                window = list(grid[range(row, row+4), range(col, col+4)])
                if (window.count(piece) == 3 and window.count(0) == 1):
                    my_3 += 1
                elif (window.count(piece%2+1) == 3 and window.count(0) == 1):
                    op_3 += 1
                elif (window.count(piece) == 4 and window.count(0) == 0):
                    my_4 += 1
                elif (window.count(piece%2+1) == 4 and window.count(0) == 0):
                    op_4 += 1
        # negative diagonal
        for row in range(3, 6):
            for col in range(4):
                window = list(grid[range(row, row-4, -1), range(col, col+4)])
                if (window.count(piece) == 3 and window.count(0) == 1):
                    my_3 += 1
                elif (window.count(piece%2+1) == 3 and window.count(0) == 1):
                    op_3 += 1
                elif (window.count(piece) == 4 and window.count(0) == 0):
                    my_4 += 1
                elif (window.count(piece%2+1) == 4 and window.count(0) == 0):
                    op_4 += 1

        is_term = ((my_4 != 0) or (op_4!=0))  

        return is_term,my_3,my_4,op_3,op_4 


    # Helper function for minimax: calculates value of heuristic for grid
    def get_heuristic(my_3,my_4,op_3,op_4):        
        score = 10*my_3 - 15*op_3 - 1e4*op_4 + 1e6*my_4
        return score

    # Uses minimax to calculate value of dropping piece in selected column
    def score_move(grid, col, mark, config, nsteps):
        next_grid = drop_piece(grid, col, mark, config)
        score = minimax(next_grid, nsteps-1, False, mark, config)
        return score

    # Minimax implementation with alfabeta pruning
    def minimax(node, depth, maximizingPlayer, mark):
        is_term,my_3,my_4,op_3,op_4 = createNcheck_windows(node,mark)#mark being my piece
        is_terminal = is_term
        valid_moves = [c for c in range(COLUMNS) if node[0][c] == 0]
        if is_terminal:
            value = get_heuristic(my_3,my_4,op_3,op_4)
            if value > 1e5:
                value= 1e5+ 1e5*(depth)
            if value < - 1e3:
                value= -(1e4- 1e3*(depth))
            return value
        if depth == 0  or len(valid_moves)== 0 :
            return get_heuristic(my_3,my_4,op_3,op_4)
            

        if maximizingPlayer:
            value = -np.Inf

            for col in valid_moves:
                child = drop_piece(node, col, mark, config)
                miniv =minimax(child, depth-1, False, mark, config)
                value = max(value, miniv)
                if value > 1e5:##ALFA BETA PRUNING
                    winning_depth = (value -1e5)%1e5
                    if winning_depth>=  (depth-1):
                        break

            return value
        else:
            value = np.Inf
            for col in valid_moves:
                child = drop_piece(node, col, mark%2+1, config)
                miniv = minimax(child, depth-1, True, mark, config)
                value = min(value,miniv )
    #                 if value<=-1e3:##ALFA BETA PRUNING no alfa beta pruning fighting to the end (furthest away poss.)
    # #                     winning_depth = (value +1e4)%1e3
    # #                     if winning_depth>=  (depth-1):
    # #                         break

            return value


    # Your code here: Amend the agent!
    # Get list of valid moves
    valid_moves = [c for c in range(COLUMNS) if obs.board[c] == 0]
    # Convert the board to a 2D grid
    grid = np.asarray(obs.board).reshape(ROWS, COLUMNS)
    # Use the heuristic to assign a score to each possible board in the next step
    #CHANGED FROM 3:
    N_STEPS=2 #3       
   

    scores = dict(zip(valid_moves, [score_move(grid, col, obs.mark, config, N_STEPS) for col in valid_moves]))
    # Get a list of columns (moves) that maximize the heuristic
    max_cols = [key for key in scores.keys() if scores[key] == max(scores.values())]
    
    
    # Select at random from the maximizing columns 
    return max_cols[len(max_cols)//2]